When predicting a large number of classes it often becomes inefficient to train a generalized linear model for each distinct class (i.e. the parameters for each softmax node at the end of a neural network). On proposed solution has been to use offline clustering of output label representations to create binary codes for each label and then use log(G) distinct sigmoid outputs at the end of a neural network (see "A Scalable Hierarchical Distributed Language Model" by Hinton and Mnih). 

Why not instead learn an optimal encoding of the output labels interleaved with training the predictive model? Here we're experimenting with assigning initial random codes, training a multi-output linear regression, and then finding better output codes by averaging the predicted output codes of samples associated with each class. 

In [1]:
import sklearn

In [2]:
data = sklearn.datasets.fetch_20newsgroups_vectorized()

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/sklearn/datasets/twenty_newsgroups.py:89: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn("Downloading dataset from %s (14 MB)", URL)


In [3]:
data

{'data': <11314x130107 sparse matrix of type '<class 'numpy.float64'>'
 	with 1787565 stored elements in Compressed Sparse Row format>,
 'target': array([17,  7, 10, ..., 14, 12, 11]),
 'target_names': ['alt.atheism',
  'comp.graphics',
  'comp.os.ms-windows.misc',
  'comp.sys.ibm.pc.hardware',
  'comp.sys.mac.hardware',
  'comp.windows.x',
  'misc.forsale',
  'rec.autos',
  'rec.motorcycles',
  'rec.sport.baseball',
  'rec.sport.hockey',
  'sci.crypt',
  'sci.electronics',
  'sci.med',
  'sci.space',
  'soc.religion.christian',
  'talk.politics.guns',
  'talk.politics.mideast',
  'talk.politics.misc',
  'talk.religion.misc']}

In [4]:
X = data['data']; Y = data['target']

In [10]:
model = sklearn.linear_model.Ridge()

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=False).fit(X)
X_tfidf = tfidf.transform(X)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [19]:
n_samples = len(Y)
n_labels = len(set(Y))
n_bits = int(np.ceil(np.log2(n_labels)))

In [28]:
# generate random binary codes for each output labels
output_codes = {}
 
for output_label in set(Y):
    candidate_code = tuple((np.random.randn(n_bits) > 0).astype(float))
    while candidate_code in output_codes.values():
        candidate_code = tuple((np.random.randn(n_bits) > 0).astype(float))
    output_codes[output_label] = candidate_code

In [29]:
output_codes

{0: (0.0, 0.0, 1.0, 1.0, 1.0),
 1: (1.0, 0.0, 0.0, 1.0, 0.0),
 2: (0.0, 1.0, 1.0, 0.0, 0.0),
 3: (1.0, 0.0, 1.0, 1.0, 0.0),
 4: (0.0, 1.0, 0.0, 0.0, 1.0),
 5: (1.0, 1.0, 0.0, 1.0, 0.0),
 6: (1.0, 1.0, 1.0, 0.0, 1.0),
 7: (1.0, 0.0, 1.0, 0.0, 1.0),
 8: (1.0, 1.0, 0.0, 1.0, 1.0),
 9: (1.0, 1.0, 1.0, 1.0, 0.0),
 10: (1.0, 0.0, 0.0, 0.0, 1.0),
 11: (1.0, 0.0, 0.0, 0.0, 0.0),
 12: (0.0, 1.0, 1.0, 1.0, 1.0),
 13: (0.0, 0.0, 0.0, 1.0, 1.0),
 14: (0.0, 0.0, 1.0, 0.0, 1.0),
 15: (1.0, 0.0, 1.0, 1.0, 1.0),
 16: (1.0, 1.0, 1.0, 1.0, 1.0),
 17: (0.0, 1.0, 1.0, 1.0, 0.0),
 18: (1.0, 0.0, 1.0, 0.0, 0.0),
 19: (0.0, 0.0, 1.0, 1.0, 0.0)}

In [30]:
encoded_Y = np.array([output_codes[yi] for yi in Y])

In [31]:
encoded_Y

array([[ 0.,  1.,  1.,  1.,  0.],
       [ 1.,  0.,  1.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  1.,  0.,  1.],
       [ 0.,  1.,  1.,  1.,  1.],
       [ 1.,  0.,  0.,  0.,  0.]])

In [32]:
model.fit(X_tfidf, encoded_Y)

KeyboardInterrupt: 